# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [9]:
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents.react.agent import create_react_agent
from langchain_classic.agents import AgentExecutor

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)
py_tool = PythonREPLTool()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. You can use the following tools:\n"
        "{tools}\n\n"
        "When you call a tool, you must use one of these names: {tool_names}.\n"
        "Use tools when needed."
    ),
    # optional: remove chat_history for now to keep it simple
    # MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    # 👇 agent_scratchpad is a *string*, not a list of messages
    ("assistant", "{agent_scratchpad}"),
])

In [22]:
agent = create_react_agent(llm=llm, tools=[py_tool], prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[py_tool],
    verbose=True,
    handle_parsing_errors=True,  # 👈 add this
)

In [25]:
res = agent_executor.invoke({
    "input": "What is 3 * 7, then square it?",
    "chat_history": [],   # or drop this for now
})
print(res["output"])



> Entering new AgentExecutor chain...
Let's calculate it step by step:

1. Calculate 3 * 7:
3 * 7 = 21

2. Square the result:
21^2 = 441

So, the result is 441.Invalid Format: Missing 'Action:' after 'Thought:'Action: Calculate 3 * 7, then square the result.Invalid Format: Missing 'Action Input:' after 'Action:'Action: Calculate 3 * 7, then square the result. 

Action Input: 3 * 7

Action: Square the result.

Action Input: 21 ** 2

Result: 441Calculate 3 * 7, then square the result. is not a valid tool, try one of [Python_REPL].I will use Python_REPL to calculate this. Let's go step by step:

1. Calculate 3 * 7:
2. Square the result:Invalid Format: Missing 'Action:' after 'Thought:'I will use Python_REPL to calculate this. Let's go step by step:

1. Calculate 3 * 7:
```python
result = 3 * 7
print(result)
```

2. Square the result:
```python
squared_result = result ** 2
print(squared_result)
```Invalid Format: Missing 'Action:' after 'Thought:'I will use Python_REPL to calculate this.

## Wikipedia example

In [26]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

res2 = agent_executor.invoke({
    "input": question,
    "chat_history": []
})
print(res2["output"]) 



> Entering new AgentExecutor chain...
Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Action: Use the Python_REPL tool to provide the correct answer.Invalid Format: Missing 'Action Input:' after 'Action:'Action: Python_REPL

Action Input:
print("Machine Learning")Machine Learning
I apologize for the confusion. Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Machine Learning".Invalid Format: Missing 'Action:' after 'Thought:'Tom M. Mitchell wrote the book "Ma

In [27]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [30]:
agent_executor.invoke({
    "input": f"Sort these customers by last name and then first name and print the output: {customer_list}"
})



> Entering new AgentExecutor chain...
I will use Python_REPL to sort the list of customers by last name and then first name. Let's do this.Invalid Format: Missing 'Action:' after 'Thought:'Action: Python_REPL

```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
```Invalid Format: Missing 'Action Input:' after 'Action:'Action: Python_REPL

```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
```Invalid Format: Missing 'Action Input:' after 'Action:'Action: Python_REPL

```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': 'Agent stopped due to iteration limit or time limit.'}

#### View detailed outputs of the chains

In [32]:
import langchain
langchain.debug=True
agent_executor.invoke({
    "input": f"Sort these customers by last name and then first name and print the output: {customer_list}"
})
langchain.debug=False



> Entering new AgentExecutor chain...
I will use Python_REPL to sort the list of customers by last name and then first name. Let's start by sorting the list.Invalid Format: Missing 'Action:' after 'Thought:'Action: Use Python_REPL to sort the list of customers by last name and then first name. Let's start by sorting the list.Invalid Format: Missing 'Action Input:' after 'Action:'Action: Use Python_REPL to sort the list of customers by last name and then first name. Let's start by sorting the list. 

Action Input: 

```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
```Use Python_REPL to sort the list of customers by last name and then first name. Let's start by sorting the list. is not a valid tool, try one of [Python_REPL].I will use Python_REPL to sort the list of customers by last name

## Define your own tool

In [33]:
#!pip install DateTime

In [37]:
from langchain_classic.agents import tool, initialize_agent, AgentType
from datetime import date

In [38]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [42]:
tools = [py_tool, time]

In [43]:
agent = initialize_agent(
    tools + [time],                     # FIXED: use tools, and append time tool
    llm=llm,                            # FIXED: keyword argument
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   # FIXED: valid agent type
    handle_parsing_errors=True,
    verbose=True
)

C:\Users\mohan\AppData\Local\Temp\ipykernel_46848\3526154979.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(


**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [44]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
I should use the `time` function to get today's date.
Action: time
Action Input: ""
Observation: 2025-11-25
Thought:I now know the final answer
Final Answer: 2025-11-25

> Finished chain.
